In [1]:
import faiss
from faiss import write_index, read_index
from sentence_transformers import SentenceTransformer
import math
import pandas as pd
import requests
from urllib.parse import urlencode
import os
import numpy as np
from catboost import CatBoostRanker, Pool
from functools import partial
import json
import pyarrow.parquet as pq
import time
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import requests
from requests.exceptions import HTTPError
import json

In [2]:
queries = pd.read_csv(
    'test_dataset_submission_queries.csv'
)

In [3]:
def t9(text: str) -> str:
    speller_url = r"https://speller.yandex.net/services/spellservice.json/checkText"
    try:
        response = requests.get(speller_url, params={'text' : text})
    except HTTPError as e:
        print(f"An HTTP error has occured: {e}")
        return ""

    decoded: list = json.loads(response.text)

    output = text

    for error in decoded:
        pos = len(output) - (len(text) - error['pos'])
        slen = error['len']
        repl = error['s'][0]

        output = output[:pos] + repl + output[pos+slen:]

    return output

for index, query in queries.iterrows():
    query["query"] = t9(query["query"]).lower()
print(queries.head())
queries = queries.drop_duplicates()

                                               query
0  битва сильнейших экстрасенсов 2023 смотреть | ...
1  битва сильнейших экстрасенсов 2023\битва сильн...
2            экстрасенсы. битва сильнейших, 4 выпуск
3                                 супер стар 4 сезон
4                                пять ночей с фредди


In [4]:
st_model = SentenceTransformer(
    'cointegrated/rubert-tiny2',
    device='cuda'
)

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

.gitignore:   0%|          | 0.00/6.00 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

tinybert-ru-labse-adapter-v2.pt:   0%|          | 0.00/963k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
candidates = pd.read_parquet(
    'videos.parquet',
    engine='fastparquet',
    columns=['video_id', 'video_title']
)

automarkup = pd.read_parquet(
    'automarkup.parquet',
    engine='fastparquet'
)

automarkup = automarkup[~automarkup['query'].isna()]
automarkup['query'] = automarkup['query'].apply(lambda x: x.lower())

In [6]:
with open('ind2videoid.json', 'r') as f:
        ind2videoid = json.load(f)

In [7]:
print(type(queries))
queries = queries["query"].tolist()
qembeddings = st_model.encode(
    queries,
    batch_size=100,
    show_progress_bar=True
)

search_cpu_index = read_index('candidates.index')
search_cpu_index.is_trained, search_cpu_index.ntotal

ind2videoid = pd.DataFrame.from_dict(ind2videoid, orient='index')

<class 'pandas.core.frame.DataFrame'>


Batches:   0%|          | 0/18 [00:00<?, ?it/s]

In [27]:
topk = 5
distance, faiss_ind = search_cpu_index.search(qembeddings, topk)

generated_cand = {
    'query': [],
    'video_id': []
}

for i, q in enumerate(queries):
    print(f"Epoch {i}")
    vids = faiss_ind[i]
    for v in vids:
        vid = ind2videoid.iloc[v,0]
        generated_cand['video_id'].append(vid)
    for j in range(len(vids)):
        generated_cand['query'].append(q)
    
generated_cand = pd.DataFrame(generated_cand)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110


In [48]:
generated_cand['target'] = np.nan
generated_cand['target'] = generated_cand['target'].fillna(0)
candidates_with_target = generated_cand
print(generated_cand.columns)
print(generated_cand)

Index(['query', 'video_id', 'target'], dtype='object')
                                                   query  ... target
0      битва сильнейших экстрасенсов 2023 смотреть | ...  ...    0.0
1      битва сильнейших экстрасенсов 2023 смотреть | ...  ...    0.0
2      битва сильнейших экстрасенсов 2023 смотреть | ...  ...    0.0
3      битва сильнейших экстрасенсов 2023 смотреть | ...  ...    0.0
4      битва сильнейших экстрасенсов 2023 смотреть | ...  ...    0.0
...                                                  ...  ...    ...
25900               мужское женское выпуск от 04 06 2021  ...    0.0
25901               мужское женское выпуск от 04 06 2021  ...    0.0
25902               мужское женское выпуск от 04 06 2021  ...    0.0
25903               мужское женское выпуск от 04 06 2021  ...    0.0
25904               мужское женское выпуск от 04 06 2021  ...    0.0

[25905 rows x 3 columns]


In [29]:
features_parquet = pq.ParquetFile('features.parquet')
features, filter_date = None, '2023-10-02'

for batch in features_parquet.iter_batches():
    tmp = batch.to_pandas()
    if features is None:
        features = tmp[tmp['report_date'] >= filter_date]
    else:
        features = pd.concat([
            features, 
            tmp[tmp['report_date'] == filter_date]
        ], axis=0)

features = features.drop(
    [
        'v_channel_reg_datetime',
        'v_channel_type',
        'v_pub_datetime'
    ], 
    axis=1
)

In [70]:
full_df = candidates_with_target.merge(
    features,
    how='inner',
    left_on='video_id',
    right_on='video_id'
)
"""del features"""
print(full_df)
full_df = full_df.drop('report_date', axis=1)
full_df = full_df.drop_duplicates()
print(full_df)

                                                      query  ... v_cr_click_comment_1_days
0         битва сильнейших экстрасенсов 2023 смотреть | ...  ...                       0.0
1         битва сильнейших экстрасенсов 2023\битва сильн...  ...                       0.0
2         битва сильнейших экстрасенсов 2023\битва экстр...  ...                       0.0
3         битва сильнейших экстрасенсов\битва экстрасенс...  ...                       0.0
4         битва сильнейших экстрасенсов 2023 смотреть\би...  ...                       0.0
...                                                     ...  ...                       ...
11314993                                                NaN  ...                       0.0
11314994                                                NaN  ...                       0.0
11314995                                                NaN  ...                       0.0
11314996                                                NaN  ...                       0.0

In [31]:
full_df['group_id'] = full_df.groupby(['query']).ngroup()
for group in full_df['group_id'].unique():
    part_df = full_df[full_df['group_id'] == group]
print(full_df)

                                                   query  ... group_id
0      битва сильнейших экстрасенсов 2023 смотреть | ...  ...      232
1      битва сильнейших экстрасенсов 2023\битва сильн...  ...      238
2      битва сильнейших экстрасенсов 2023\битва экстр...  ...      239
3      битва сильнейших экстрасенсов\битва экстрасенс...  ...      253
4      битва сильнейших экстрасенсов 2023 смотреть\би...  ...      236
...                                                  ...  ...      ...
14963                               шоу вована и лексуса  ...     1597
14964                               шоу вована и лексуса  ...     1597
14965               мужское женское выпуск от 04 06 2021  ...      809
14966               мужское женское выпуск от 04 06 2021  ...      809
14967               мужское женское выпуск от 04 06 2021  ...      809

[14968 rows x 28 columns]


In [32]:
print(full_df.head)
print(full_df.columns)
full_df = full_df.sort_values('group_id')

<bound method NDFrame.head of                                                    query  ... group_id
0      битва сильнейших экстрасенсов 2023 смотреть | ...  ...      232
1      битва сильнейших экстрасенсов 2023\битва сильн...  ...      238
2      битва сильнейших экстрасенсов 2023\битва экстр...  ...      239
3      битва сильнейших экстрасенсов\битва экстрасенс...  ...      253
4      битва сильнейших экстрасенсов 2023 смотреть\би...  ...      236
...                                                  ...  ...      ...
14963                               шоу вована и лексуса  ...     1597
14964                               шоу вована и лексуса  ...     1597
14965               мужское женское выпуск от 04 06 2021  ...      809
14966               мужское женское выпуск от 04 06 2021  ...      809
14967               мужское женское выпуск от 04 06 2021  ...      809

[14968 rows x 28 columns]>
Index(['query', 'video_id', 'target', 'v_category', 'total_comments',
       'v_year_views

In [33]:
metainfo_columns = ['query', 'video_id', 'target', 'group_id', 'v_category']
g_data = full_df['group_id']
train_df = full_df.drop(metainfo_columns, axis=1)


In [34]:
data = Pool(
    data=train_df.values,
    group_id=g_data.values,
    feature_names=train_df.columns.to_list()
)

In [37]:
model = CatBoostRanker()

model.load_model('ranker.ckpt')

In [39]:
predictions = model.predict(data, verbose=True)
print(type(predictions))

<class 'numpy.ndarray'>


In [54]:
print(len(predictions))
for index, obj in full_df.iterrows(): 
    full_df.loc[index,['target']] = predictions[index]

14968
[-0.00901273790191226, -0.00860482935206619, -0.005088042095195533, -0.008379563423318359, -0.00881190164329837, -0.0009625468183972197, -0.008128431851417872, -0.00893171833388351, -0.00834948757744769, -0.009139395417378182, -0.008554921834615925, -0.008483168742145608, 0.3571117009705631, -0.00893171833388351, -0.008459103754375974, -0.00837969311606912, -0.0058633987414351135, -0.008554921834615925, -0.00811497146779061, -0.008511687269617596, -0.008498555472158387, 0.2370937715084631, -0.007564643931230464, -0.008510078048743708, -0.008579252011770677, -0.008430253715005165, -0.008579252011770677, 0.29402351149204264, -0.008579252011770677, 0.08205897103125537, -0.008389212500901522, -0.009088122512935115, -0.0084668843588247, -0.008930184557470966, -0.008673478431546755, -0.008691748476280736, -0.00432555726928141, -0.008366726858525908, -0.007836839192325584, -0.0074265536948114885, 0.0007482752569035744, -0.007536227004264262, -0.008192727909403175, -0.00838789211331914, 

In [67]:
pre_submission = full_df.sort_values(by=["target"], ascending=False)

In [72]:
buffer = pre_submission.drop(
    axis=1,
    labels=[
        "v_category",
        "target",
        "total_comments",
        "v_year_views",
        "v_month_views",
        "v_week_views",
        "v_day_views",
        "v_likes",
        "v_dislikes",
        "v_duration",
        "v_cr_click_like_7_days",
        "v_cr_click_dislike_7_days",
        "v_cr_click_vtop_7_days",
        "v_cr_click_long_view_7_days",
        "v_cr_click_comment_7_days",
        "v_cr_click_like_30_days",
        "v_cr_click_dislike_30_days",
        "v_cr_click_vtop_30_days",
        "v_cr_click_long_view_30_days",
        "v_cr_click_comment_30_days",
        "v_cr_click_like_1_days",
        "v_cr_click_dislike_1_days",
        "v_cr_click_vtop_1_days",
        "v_cr_click_long_view_1_days",
        "v_cr_click_comment_1_days",
        "group_id"
    ]
)

In [73]:
print(generated_cand.columns)

submission = buffer.to_csv("submission.csv", columns= ['query', 'video_id'])
df1 = pd.read_csv('submission.csv')
df1 = df1.iloc[:, 1:]
df1.to_csv('submission_trial.csv', index=False)

Index(['query', 'video_id', 'target'], dtype='object')
